<a href="https://colab.research.google.com/github/bikash-bhoi/eva4/blob/master/Session12/Session_12_TinyImgNet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##Clone Github Directory to import packages
%%bash
mkdir temp
git clone https://github.com/bikash-bhoi/eva4.git temp
if [ ! -d ./models ]; then
    mkdir models
fi
if [ ! -d ./utils ]; then
    mkdir utils
fi
cp -r temp/models/* ./models
cp -r temp/utils/* ./utils
rm -rf temp

Cloning into 'temp'...


In [0]:
#!pip install pytorch-gradcam
!pip install albumentations
!pip install livelossplot
#!pip install --no-cache-dir git+https://github.com/NVIDIA/apex

In [0]:
#from utils.download_load import download_load
#from utils.display_imgs import display_imgs
from utils.train_test import train , test, predict, get_misclassified
from utils.disp_summary import disp_summary
#from utils.gradcam import grad_cam, gradcam_plot
from models.resnet import ResNet18,ResNet
from utils.range_test import lr_range_test
from utils.train_model import train_model
#from utils.zig_zag_plot import zigzag_plot

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchsummary
from torchsummary import summary
import torch,os
import torch.utils.data as data

In [0]:
!rm -rf /content/tiny-imagenet-200

In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
#!rm -rf /content/tiny-imagenet-200
!unzip -q "/content/gdrive/My Drive/tiny-imagenet-200.zip"


Mounted at /content/gdrive


In [0]:
from shutil import move
data_dir = '/content/tiny-imagenet-200/'
train_dir = data_dir+'train/'
val_dir = data_dir+'val/'
val_annot = val_dir+'val_annotations.txt'

print('Pre')
print(len(os.listdir(val_dir+'images/')))
print(sum(1 for line in open(val_annot)))


for tp in os.listdir(train_dir):
    
    for i in range(385,500):
        img=train_dir+tp+'/'+'images/'+tp+'_'+str(i)+'.JPEG'
        # Move 113 images to Val dataset
        move(img,val_dir+'images/')
    #Move annotation txt to val annotation
    train_annot = train_dir+tp+'/'+tp+'_boxes.txt'
    
    #Read Train annotaions
    with open(train_annot) as f1:lines = f1.readlines()

    #Append last 115 Train annotations to val annotation
    with open(val_annot, 'a') as f2: f2.writelines(lines[385:])

    #Overwrite Train annotation with first 385 Lines
    with open(train_annot, 'w') as f3: f3.writelines(lines[:385])

print('Post')
print(len(os.listdir(val_dir+'images/')))
print(sum(1 for line in open(val_annot)))


Pre
10000
10000
Post
33000
33000


In [0]:
data_dir = '/content/tiny-imagenet-200/'
num_workers = {'train' : 100,'val'   : 0,'test'  : 0}
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(20),
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
    ])
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) 
                  for x in ['train', 'val','test']}
dataloaders = {x: data.DataLoader(image_datasets[x], batch_size=512, shuffle=True, num_workers=num_workers[x])
                  for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

In [0]:
dataset_sizes

{'test': 10000, 'train': 77000, 'val': 33000}

In [0]:
import torchvision.models as models
model = models.resnet18()
#Finetune Final few layers to adjust for tiny imagenet input
model.conv1 = nn.Conv2d(3,64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False)
model.maxpool = nn.Sequential()
model.avgpool = nn.AdaptiveAvgPool2d(1)
model.fc.out_features = 200

#model=ResNet18(200)
#model.avgpool = nn.AdaptiveAvgPool2d(1)

use_cuda= torch.cuda.is_available()
device=torch.device('cuda' if use_cuda else 'cpu')
model=model.to(device)

In [0]:
disp_summary(model,[3,64,64])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
            Conv2d-4           [-1, 64, 64, 64]          36,864
       BatchNorm2d-5           [-1, 64, 64, 64]             128
              ReLU-6           [-1, 64, 64, 64]               0
            Conv2d-7           [-1, 64, 64, 64]          36,864
       BatchNorm2d-8           [-1, 64, 64, 64]             128
              ReLU-9           [-1, 64, 64, 64]               0
       BasicBlock-10           [-1, 64, 64, 64]               0
           Conv2d-11           [-1, 64, 64, 64]          36,864
      BatchNorm2d-12           [-1, 64, 64, 64]             128
             ReLU-13           [-1, 64, 64, 64]               0
           Conv2d-14           [-1, 64,

In [0]:
lrs = [j*(10**i) for i in range(-3,-1) for j in range(1,11)]
lr_range_test(lrs,model,device,train_loader, test_loader)


In [0]:

train_loader = dataloaders['train']
test_loader = dataloaders['val']

optimizer = optim.SGD(model.parameters(), lr=0.01,  momentum=0.9)
scheduler = OneCycleLR(optimizer, max_lr=0.03,  total_steps=50,pct_start=0.3, final_div_factor=1, div_factor=10)
lrs=[]

for epoch in range(1, 50):
    curr_lr=optimizer.param_groups[0]['lr']
    lrs.append(curr_lr)
    print(f'Epoch: {epoch} Learning_Rate {curr_lr}')
    train_acc1 = train(model, device, train_loader, optimizer, epoch)
    test_acc1 = test(model, device, test_loader)
    print('Test acc:', test_acc1)
    scheduler.step()


  0%|          | 0/151 [00:00<?, ?it/s]

Epoch: 1 Learning_Rate 0.002999999999999999


  0%|          | 0/151 [00:00<?, ?it/s]


Test set: Average loss: -31.2994, Accuracy: 0/33000 (0.00%)

Test acc: 0.0
Epoch: 2 Learning_Rate 0.0033384731855453827


Loss=-201.3048095703125 Batch_id=150 Accuracy=1.10: 100%|██████████| 151/151 [02:45<00:00,  1.09s/it]
  0%|          | 0/151 [00:00<?, ?it/s]


Test set: Average loss: -187.9816, Accuracy: 0/33000 (0.00%)

Test acc: 0.0
Epoch: 3 Learning_Rate 0.004336920283317343


Loss=-1681.71337890625 Batch_id=150 Accuracy=0.94: 100%|██████████| 151/151 [02:43<00:00,  1.08s/it]
  0%|          | 0/151 [00:00<?, ?it/s]


Test set: Average loss: -1420.3485, Accuracy: 0/33000 (0.00%)

Test acc: 0.0
Epoch: 4 Learning_Rate 0.0059452749866815945


Loss=-30797.21484375 Batch_id=150 Accuracy=0.83: 100%|██████████| 151/151 [02:41<00:00,  1.34it/s]

KeyboardInterrupt: ignored

In [0]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001,  momentum=0.9)
scheduler = OneCycleLR(optimizer, max_lr=0.03,  total_steps=50,pct_start=0.3, final_div_factor=1, div_factor=10)
save_path = '/content/gdrive/My Drive/Colab Notebooks/eva4_s12/'
train_model(save_path,model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs=50, scheduler=scheduler)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 1/50
----------


  0%|          | 0/151 [00:00<?, ?it/s]

Train Loss: -11.1578 Acc: 0.0101
Val Loss: -38.4898 Acc: 0.0000

Epoch 2/50
----------


  0%|          | 0/151 [00:00<?, ?it/s]

...

Train Loss: -43091.1595 Acc: 0.0112
Val Loss: -166963.8848 Acc: 0.0000

Epoch 5/50
----------


  0%|          | 0/151 [00:00<?, ?it/s]

Train Loss: -2621269.2963 Acc: 0.0074
Val Loss: -8614144.5176 Acc: 0.0000

Epoch 6/50
----------


Loss=-671684096.0 Batch_id=127 :  85%|████████▍ | 128/151 [02:21<00:20,  1.14it/s]

KeyboardInterrupt: ignored

In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
PATH = '/content/gdrive/My Drive/Colab Notebooks/eva4_s9/assignment11_ResNetcust_model.h5'
!touch PATH
torch.save(model.state_dict(), PATH)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
